In [20]:
from sklearn import datasets
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from os import walk
import cv2
import numpy as np
from sklearn.externals import joblib
import os
import csv

In [23]:
def getFish(filepath,family):
    img = cv2.imread(filepath)
    # standardize size for prediction step
    img = cv2.resize(img, (150, 150))
    img = img/255.0
    img = img.reshape(img.shape[0]*img.shape[1]*img.shape[2])
    
    if "noeyespot" in filepath:
        classification = "noeyespot"
    else:
        classification = "eyespot"  
    
    _, _, species = filepath.partition(family+'/')
    
    #res = model.predict(img)
    return(img, classification, species)

def create_x_y(image_filepaths):
    
    #create lists for image training data and label training data
    x_train = []
    y_train = []
    
    #iterate through list of image filepaths
    #get pixel, label data and store in x,y
    #append x and y to x_train and y_train lists
    #convert classification to binary
    for i in range(len(image_filepaths)):
        x, y, _ = getFish(image_filepaths[i])
        x_train.append(x)
        y_train.append(y)
        if (y_train[i] == 'eyespot'):
            y_train[i] = 0
        else:
            y_train[i] = 1
        
    #convert labels back to categorical from binary
    #y_train = to_categorical(y_train)
    
    return x_train, y_train

def predict(model, get_fish_output, eyespot_threshold):
    
    img, correct_class, fam = get_fish_output
    img = img.reshape((1,) + img.shape)
    
    weights = clf.predict_proba(img)
    
    y_pred = 2
    prediction =''

    if weights [:,0][0] > eyespot_threshold:
        y_pred = 0
    else: 
        y_pred = 1
    
    if (y_pred == 0):
        prediction = "eyespot"
    else:
        prediction = "noeyespot"

    #print "Model classification:",prediction,"\nCorrect classification:", correct_class,"\nPrediction wieghts:",prediction_weights,"\nType:",fam,"\n"
    return(prediction, correct_class, weights, fam)

def get_img_paths(family_filepath):
    
    fish = []
    files=[]

    for (dirpath, dirnames, filenames) in walk(family_filepath):
        for i in range(len(filenames)):
            img = dirpath + filenames[i]
            files.append(filenames[i])
            if "Store" not in img and ".csv" not in img:  
                fish.append(img)
        break
    
    return fish, files

# Load Model

In [27]:
os.chdir('/Users/leannwoo/machine_learning_fishes')
clf = joblib.load('rf1.pkl') 

In [28]:
tobeclassified = 'Labridae2'

family_path = "/Users/leannwoo/Dropbox/machine_learning_fishes/randall/Labridae2/"

fish_list_img, fish_img_names = get_img_paths(family_path)

os.chdir(family_path)

In [29]:
with open('classification_data.csv', 'wb') as csvfile:
    filewriter = csv.writer(csvfile, delimiter=',',
                            quotechar='|', quoting=csv.QUOTE_MINIMAL)
    filewriter.writerow(['Model Classification', 'Prediction Weights','Species'])
    
    try:
        os.mkdir('eyespot')
        os.mkdir('noeyespot')
    except OSError:
        1
      #  print "unsuccessful"

    for i in range(len(fish_list_img)):
        classification,_,weights,family = predict(clf, getFish(fish_list_img[i],tobeclassified), 0.4)
        #species = family.split('_')[1]
        filewriter.writerow([classification, weights, family])
        
        #weights = np.array2string(weights)
        #weights = (weights.split('[['))[1].split(']]')[0]
        
        
        if (classification == 'eyespot'):
            try:
                os.rename(fish_img_names[i], 'eyespot/' + fish_img_names[i])
            except OSError:
                1
        else:
            try:
                os.rename(fish_img_names[i], 'noeyespot/' + fish_img_names[i])
            except OSError:
                1